# Example of using LLM-based information retrieval for recommendation task

Run in google colaboratory

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import sys
import os

repo_path = '/content/drive/MyDrive/Colab Notebooks/thesis_work/llm-for-rec'
sys.path.append(repo_path)

In [3]:
%cd '{repo_path}'

/content/drive/MyDrive/Colab Notebooks/thesis_work/llm-for-rec


## Install requirements

In [4]:
!pip install -q -r '{repo_path}/requirements/requirements.txt'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.6/973.6 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 73.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.4/310.4 kB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 57.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.1/324.1 kB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 58.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 64.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 kB 25.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.8/124.8 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 15.2 

## Get dataset and config

In [5]:
from llm4rec.tasks import RetrievalRecommender
from llm4rec.evaluation.evaluate import evaluate_pipeline
from copy import deepcopy
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv
from llm4rec.utils import EmbeddingsFromFile

In [6]:
from llm4rec.pipelines import RecBolePipelineRecommender
from llm4rec.dataset import RecboleSeqDataset
from llm4rec.evaluation.trainer import PipelineTrainer
from llm4rec.utils.dataset_utils import ml100k_preprocess
from recbole.data.utils import data_preparation
from recbole.config import Config
import os
import torch


model_cls = RecBolePipelineRecommender
dataset_name = 'ml-100k'

config = Config(model=model_cls, dataset=dataset_name,
             config_file_list=['./llm4rec/configs/dataset_ml100k.yaml',
                               './llm4rec/configs/overall.yaml'])

dataset = RecboleSeqDataset(config, preprocess_text_fn=ml100k_preprocess)
train_data, _, test_data = data_preparation(config, deepcopy(dataset))

In [7]:
config.final_config_dict['topk'] = [10, 20, 50, 100, 300]

## Case 1: open source information retrieval using HuggingFace Sentence Transformers model

Create retrieval from dataset meta information

In [8]:
retrieval = RetrievalRecommender(
                embeddings=None,
                item2text=dataset.item_token2text,
                items_info_path=os.path.join(config['data_path'], f"{config['dataset']}.item"),
                search_kwargs={'k':max(config['topk'])})

results_opensource_meta = evaluate_pipeline(config, deepcopy(dataset), tasks=[retrieval])
results_opensource_meta[2]['test_result']

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Evaluate   :   0%|                                                          | 0/118 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
Evaluate   : 100%|████████████████████████████████████████████████| 118/118 [00:20<00:00,  5.80it/s]


OrderedDict([('recall@10', 0.0286),
             ('recall@20', 0.0456),
             ('recall@50', 0.0912),
             ('recall@100', 0.1495),
             ('recall@300', 0.3351),
             ('ndcg@10', 0.0132),
             ('ndcg@20', 0.0174),
             ('ndcg@50', 0.0262),
             ('ndcg@100', 0.0357),
             ('ndcg@300', 0.0602)])

Create retrieval from ItemMemory data

In [9]:
from llm4rec.memory import ItemMemory

item_filepath = "/content/drive/MyDrive/Colab Notebooks/thesis_work/llm-for-rec/examples/develop/item_memory_summ.json"

item_memory = ItemMemory(item_ids=dataset.item_id_token[1:],
                         title_col='movie_title',
                         dataset_info_map=dataset.item_token2attr,
                         load_filename=item_filepath)

100%|██████████| 1682/1682 [00:00<00:00, 764418.61it/s]


In [10]:
retrieval = RetrievalRecommender(
                embeddings=None,
                item2text=lambda x: item_memory.retrieve(x, retr_type=''),
                item_memory=item_memory,
                load_from_file=False,
                items_info_path=os.path.join(config['data_path'], f"{config['dataset']}.item"),
                search_kwargs={'k':max(config['topk'])})

results_opensource_memory = evaluate_pipeline(config, deepcopy(dataset), tasks=[retrieval])
results_opensource_memory[2]['test_result']

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Evaluate   : 100%|████████████████████████████████████████████████| 118/118 [00:26<00:00,  4.39it/s]


OrderedDict([('recall@10', 0.0117),
             ('recall@20', 0.018),
             ('recall@50', 0.0467),
             ('recall@100', 0.088),
             ('recall@300', 0.2227),
             ('ndcg@10', 0.0057),
             ('ndcg@20', 0.0073),
             ('ndcg@50', 0.0128),
             ('ndcg@100', 0.0194),
             ('ndcg@300', 0.0375)])

Retrieval with user profile

In [15]:
from llm4rec.memory import UserMemory
from langchain_groq import ChatGroq
from dotenv import load_dotenv
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain import HuggingFaceHub
from llm4rec.tasks import UserAugmentation
import torch

user_filepath = "/content/drive/MyDrive/Colab Notebooks/thesis_work/llm-for-rec/examples/develop/user"

embedding_size = 384
embedding_fn = HuggingFaceEmbeddings(
                model_name="all-MiniLM-L6-v2", model_kwargs={"device":"cuda:0" if torch.cuda.is_available() else "cpu"})

path_to_env = os.path.join(repo_path, "api_keys.env")
load_dotenv(path_to_env)

llm = ChatGroq(model_name="llama3-70b-8192", temperature=0)
user_memory = UserMemory(train_dataset=train_data.dataset,
                         load_filename=user_filepath,
                         user_attributes=dataset.user_token2text,
                         short_term_limit=20, llm=llm,
                         embeddings=embedding_fn,
                         item_memory=item_memory,
                         emb_size=embedding_size)
retrieval = RetrievalRecommender(
                embeddings=None,
                item2text=dataset.item_token2text,
                items_info_path=os.path.join(config['data_path'], f"{config['dataset']}.item"),
                search_kwargs={'k':max(config['topk'])})
augmentation = UserAugmentation(user_memory)

tasks = [augmentation, retrieval]
results_opensource_profile = evaluate_pipeline(config, deepcopy(dataset), tasks=tasks)
results_opensource_profile[2]['test_result']

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
100%|██████████| 943/943 [00:00<00:00, 861517.90it/s]
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Evaluate   : 100%|████████████████████████████████████████████████| 118/118 [42:37<00:00, 21.67s/it]


OrderedDict([('recall@10', 0.018),
             ('recall@20', 0.0308),
             ('recall@50', 0.0689),
             ('recall@100', 0.1156),
             ('recall@300', 0.2821),
             ('ndcg@10', 0.0084),
             ('ndcg@20', 0.0116),
             ('ndcg@50', 0.0191),
             ('ndcg@100', 0.0268),
             ('ndcg@300', 0.049)])



## Case 2: OpenAI information retrieval using OpenAIEmbeddings

In [16]:
path_to_env = os.path.join(repo_path, "api_keys.env")
load_dotenv(path_to_env)

openai_api_key = os.environ.get("API_KEY")
embeddings_model = OpenAIEmbeddings(
            openai_api_key=openai_api_key, model="text-embedding-ada-002"
        )

Embeddings from dataset item meta information

In [23]:
retrieval = RetrievalRecommender(
                embeddings=embeddings_model,
                item2text=dataset.item_token2text,
                items_info_path=os.path.join(config['data_path'], f"{config['dataset']}.item"),
                search_kwargs={'k':max(config['topk'])})

results_openai_meta = evaluate_pipeline(config, deepcopy(dataset), tasks=[retrieval])
results_openai_meta[2]['test_result']

Evaluate   : 100%|████████████████████████████████████████████████| 118/118 [05:26<00:00,  2.77s/it]


OrderedDict([('recall@10', 0.0223),
             ('recall@20', 0.0467),
             ('recall@50', 0.0986),
             ('recall@100', 0.1569),
             ('recall@300', 0.3457),
             ('ndcg@10', 0.009),
             ('ndcg@20', 0.0151),
             ('ndcg@50', 0.0253),
             ('ndcg@100', 0.0348),
             ('ndcg@300', 0.0603)])

Embeddings from item memory

In [17]:
retrieval = RetrievalRecommender(
                embeddings=embeddings_model,
                item2text=lambda x: item_memory.retrieve(x, retr_type=''),
                item_memory=item_memory,
                load_from_file=False,
                items_info_path=os.path.join(config['data_path'], f"{config['dataset']}.item"),
                search_kwargs={'k':max(config['topk'])})

results_openai_memory = evaluate_pipeline(config, deepcopy(dataset), tasks=[retrieval])
results_openai_memory[2]['test_result']

Evaluate   : 100%|████████████████████████████████████████████████| 118/118 [07:24<00:00,  3.77s/it]


OrderedDict([('recall@10', 0.0201),
             ('recall@20', 0.035),
             ('recall@50', 0.0679),
             ('recall@100', 0.105),
             ('recall@300', 0.2757),
             ('ndcg@10', 0.0115),
             ('ndcg@20', 0.0152),
             ('ndcg@50', 0.0217),
             ('ndcg@100', 0.0277),
             ('ndcg@300', 0.0503)])

## Case 3: Information retrieval using ALS embeddings

In [20]:
emb = EmbeddingsFromFile(emb_file_path=os.path.join(repo_path, 'examples', 'develop','als_emb_ml100k.npz'))

In [21]:
retrieval = RetrievalRecommender(
                embeddings=emb,
                csv_loader_args=dict(csv_args={'delimiter':'\t'},
                    source_column='item_id:token',
                    metadata_columns=['movie_title:token_seq', 'release_year:token', 'class:token_seq']),
                item2text=lambda x: x,
                load_from_file=True,
                items_info_path=os.path.join(config['data_path'], f"{config['dataset']}.item"),
                search_kwargs={'k':max(config['topk'])},
                query='{user_profile}')

results_als = evaluate_pipeline(config, deepcopy(dataset), tasks=[retrieval])
results_als[2]['test_result']

Evaluate   : 100%|████████████████████████████████████████████████| 118/118 [00:11<00:00, 10.31it/s]


OrderedDict([('recall@10', 0.0445),
             ('recall@20', 0.0615),
             ('recall@50', 0.0891),
             ('recall@100', 0.1135),
             ('recall@300', 0.1262),
             ('ndcg@10', 0.0231),
             ('ndcg@20', 0.0273),
             ('ndcg@50', 0.0327),
             ('ndcg@100', 0.0367),
             ('ndcg@300', 0.0384)])

In [28]:
import pandas as pd

df_results = pd.DataFrame([results_opensource_meta[2]['test_result'], results_opensource_memory[2]['test_result'],
                           results_opensource_profile[2]['test_result'],
              results_openai_meta[2]['test_result'], results_openai_memory[2]['test_result'],
              results_als[2]['test_result']], index=['opensource_metainfo', 'opensource_memory', 'opensource_profile',
                                                   'openai_metainfo', 'openai_memory',
                                                     'als'])
df_results.to_csv('./examples/develop/inf_retr.csv')
df_results

,recall@10,recall@20,recall@50,recall@100,recall@300,ndcg@10,ndcg@20,ndcg@50,ndcg@100,ndcg@300
opensource_metainfo,0.0286,0.0456,0.0912,0.1495,0.3351,0.0132,0.0174,0.0262,0.0357,0.0602
opensource_memory,0.0117,0.0180,0.0467,0.0880,0.2227,0.0057,0.0073,0.0128,0.0194,0.0375
opensource_profile,0.0180,0.0308,0.0689,0.1156,0.2821,0.0084,0.0116,0.0191,0.0268,0.0490
openai_metainfo,0.0223,0.0467,0.0986,0.1569,0.3457,0.0090,0.0151,0.0253,0.0348,0.0603
openai_memory,0.0201,0.0350,0.0679,0.1050,0.2757,0.0115,0.0152,0.0217,0.0277,0.0503
als,0.0445,0.0615,0.0891,0.1135,0.1262,0.0231,0.0273,0.0327,0.0367,0.0384
